# Práctica 4 - AG frases en castellano con mutaciones - Lab 1
## Preparación de entorno
#### Instalar las librerías

In [47]:
# %pip install numpy

#### Importar librerías de código

In [48]:
import random
import numpy as np
import math
from collections import Counter

## Algoritmo genético SetUp

* **Algoritmos genéticos (GA)** $\rightarrow$ Clasae de algoritmos de optimización basados en los principios de la selección natural y la genética, introducidos por John Holland en 1975. El proceso de un algortimo genético puede descomponerse en los siguientes pasos:

    * **Inicialización de la población:** Se genera un conjunto inicial de soluciones candidata, que se denominan individuos o cromosomas. Cada individuo representa una posible solución al problema y puede representare como una cade binaria, una secuencia de números reales, o, en nuestro caso, una cadena de caracteres.
    
    * **Evaluación de la aptitud:** Cada individuo se evalúa mediante una función de aptitud (*fitness function*), que mide qué tan buena es la solución. 
    
    * **Selección:** Se eligen los individuos más aptos para ser padres de la siguiente generación. Entre los métodos comunes incluyen selección por ruleta, torneo y selección por ranking. Esto básicamente simula la selección natural, donde los individuos más fuertes tienen más probabilidades de reproducirse.

    * **Cruce (crossover):** Los padres seleccionados se combinan para generar nuevos individuos (hijos). Esto puede hacerse mediante técnicas como el cruce de un punto, donde se intercambian partes de los cromosomas de los padre, o cruce uniforme, dependiendo del problema.

    * **Mutación:** Se introducen cambios aleatorios en los individuos con una probabilidad dada (tasa de mutación). La mutación ayuda a mantener la diversidad en la población y evitar el estancamiento en óptimos locales, un problema común en los algoritmos genéticos.

    * **Repetición:** Los pasos de selección, cruce y mutación se repiten durante un número fijo de generaciones o hasta que se cumpla un criterio de parada, como alcanzar un nivel de aptitud deseado o un número máximo de generaciones.


<img src="media/Flow_diagram.jpg" width="30%" style="display: block; margin: 0 auto;"/>

<img src="media/Ejemplo_algoritmo_genetico.jpg" width="80%" style="display: block; margin: 0 auto; padding-top: 15px;"/>

* Aplicaciones de los algoritmos genéticos:
  * **Optimización** $\rightarrow$ Planificación de rutas (problema del viajante), asignación de recursos, optimización de funciones, etc.

  * **Inteligencia artificial** $\rightarrow$ Ajuste de hiperparámetros, entrenamiento de redes neuronales, optimización de arquitecturas.

  * **Ingeniería** $\rightarrow$ Diseño de sistemas aerodinámicos, optimización de circuitos electrónicos, diseño estructural.

  * **Bioinformática** $\rightarrow$ Alineamiento de secuencias de ADN, predicción de estructuras proteicas, análisis genético.

  * **Finanzas** $\rightarrow$ Optimización de carteras, análisis de riesgos, predicción de mercados financieros.

  * **Robótica** $\rightarrow$ Planificación de trayectorias, control de movimientos, optimización de tareas.

  * **Juegos** $\rightarrow$ Desarrollo de estrategis para juegos como el ajedrez, optimización de agenetes en videojuegos.

  * **Procesamiento de Lenguaje Natural:** Optimización de modelos de traducción, análisis de sentimientos, generación de texto.

---

En nuestro problema, cada individuo de la población es una frase candidata al objetivo. Cada cromosoma es un string cuya longitud es igual a la longitud de la frase objetivo, y cada gen es un carácter individual: letra mayúsucla o espacio.

$$\mathcal{G} = \{A, B, C, D, E, F, G, H, I, J, K, L, M, N, Ñ, O, P, Q, R, S, T, U, V, W, X, Y, Z, \hspace{5px}\}$$

El objetivo de nuestro algoritmo es que los individuos se vayan pareciendo cada vez más a la frase objetivo. Para poder calcular este parecido, vamos a definir la función de *fitness* $F_i$: $$F_i = e^{ncoin_i - ltar} - e^{-ltar}$$

donde: 
* $ncoin_i$ es el número de caracteres que coinciden entre el individuo $i$ y nuestra frase objetivo.
* $ltar$ es la longitud de nuestra frase objetivo.

Los resultados que nos puede dar esta función son: 
* $0$ $\rightarrow$ El individuo es completamente diferente a la frase objetivo.
* Cercanas a $0$ $\rightarrow$ El individuo es muy diferente a la frase objetivo.
* Cercanas a $1$ $\rightarrow$ El individuo es muy parecido a la frase objetivo.
* $1 - e^{-ltar}$ $\rightarrow$ El individuo es igual a la frase objetivo (por que $e^0 - e^{-ltar} \to 1$ cuando $ncoin_i = ltar$).

Una vez evaluamos la población inicial, lelvamos a cabo el proceso evolutivo. Este consiste en una serie de generaciones en las que la población se transforma/muta progresivamente. En cada generación, repetimos los siguientes pasos:

Primero, se identificamos el individuo más parecido al objetivo. Este será la referencia para calcular la probabilidad de replicación (generar descendencia) de los demás. Para identificarlo, calculamos el *fitness* de cada inviduo y seleccionamos el que tenga el *fitness* más alto. Este individuo se convierte en el "padre" de la siguiente generación.

A partir de esta referencia, seleccionamos individuos al azar y se decidimos si deben replicarse. Esta decisión se basa en una probabilidad proporcional al *fitness* de cada individuo: cuanto mayor es su *fitness*, mayor es la probabilidad de que genere un nuevo individuo (hijo) mediante copia y posible mutación. Este nuevo individuo sustituye a otro miembro aleatorio de la población, permitiendo que las soluciones más aptas se propaguen con mayor probabilidad a lo largo de las generaciones. La fórmula para calcular la probabilidad de replicación es:

$$Ps_i = \frac{F_i}{e^{ncoin_{max} - ltar} - e^{-ltar}}$$

Una vez hemos elegido un individuo para replicarse, generamos una copia de él, pero aplicando un proceso de mutación genética. Cada carácter de la frase (gen) tiene una probabilidad 4 de ser reemplazado por un carácter aleatorio del conjunto permitido $\mathcal{G}$. Esta transformación se expresa como:

$$gen_{mutado}=
\begin{cases}
\text{carácter aleatorio de } \mathcal{G} & \text{si } P_{random} < P_m \\
g_{original} & \text{si } P_{random} > P_m
\end{cases}$$

El proceso de mutación de un individuo (string) completo consiste en aplicar esta regla a cada uno de sus $l_{tar}$ genes, de forma independiente. Una vez hemos creado el nuevo individuo mutado, este reemplaza a otro individuo de la población elegido aleatoriamente. Esto nos permite mantener constante el tamaño total de la población y da lugar a un ciclo en el que los mejores individuos tienden a replicarse, mientras que los peores son eliminados progresivamente.

Este proceso lo repetimos durante un número determinado de generaciones. De forma periódica, analizamos el estado general de la población, calculando métricas como:
* `NTar` $\rightarrow$ Número de individuos que coinciden completamente con la frase objetivo.
* `Ptar` $\rightarrow$ Porcentaje de individuos que coinciden completamente con la frase objetivo.
* `ncoin` $\rightarrow$ Número medio de coincidencias por individuo.
* `consenso` $\rightarrow$ Individuo consenso, que se construye tomando, para cada posición, el carácter más frecuente entre todos los individuos.

Este último, el individuo consenso, es una herramienta útil para observar hacia dónde está convergiendo la población. Si el consenso coincide con la frase objetivo, podemos interpretar que la gran mayoría de la población ha alcanzado una solución correcta.

> **Nota**
> 
> Utilizamos el [Método Montecarlo](https://www.ibm.com/es-es/topics/monte-carlo-simulation) para introducir decisiones estocásticas, es decir, decisiones que no están determinadas de forma fija, sino que dependen de probabilidades. Estas decisiones las aplicamos en:
> 1. **Decisión de replicación:** A cada inviduo se le asigna una probabilida de reproducirse, proporcional a su *fitness*. Se genera un número aleatorio y, si este es menor que la probabilidad teórica, el individuo genera descendencia.
> 2. **Mutación genética:** Cada carácter del individuo replicado tiene una cierta probabilidad $P_m$ de cambiar por otro carácter aleatorio. Nuevamente, comparamos un número aleatorio con la probabilidad de mutación, lo que nos permite simular de forma sencilla y eficiente procesos evolutivos naturales de forma probabilística.

In [ ]:
# Cada gen puede tener ser cualquier letra del alfabeto (A-Z) o un espacio
GENES = list("ABCDEFGHIJKLMNÑOPQRSTUVWXYZ ")


def crear_individuo(ltar: int) -> str:
    """
    Crea un individuo (un string) de longitud ltar
    con letras aleatorias.

    Args:
        ltar (int): Longitud de la frase objetivo.

    Returns:
        str: Individuo aleatorio.
    """

    # Básicamente, mientras estemos dentro el rango de la longitud
    # de la frase objetivo, elegimos un gen (letra o espacio) al azar
    # y lo añadimos a la cadena de caracteres.
    cadena_resultado = ""

    for _ in range(ltar):
        gen = random.choice(GENES)
        cadena_resultado += gen

    return cadena_resultado


def calcular_fitness(individuo: str, frase_objetivo:str) -> tuple:
    """
    Calcula el fitness (parecido) de un individuo
    con respecto a la frase objetivo. El fitness se calcula
    como la diferencia entre el número de coincidencias
    y la longitud de la frase objetivo.

    Args:
        individuo (str): Individuo a evaluar.
        frase_objetivo (str): Frase objetivo a alcanzar.

    Returns:
        tuple: (fitness, ncoin)
            * fitness: valor entre 0 y (casi) 1 que mide lo bien que se parece el individuo al target.
            * ncoin (int): Número de coincidencias con la frase objetivo.
    """

    # Número de coincidencias entre el
    # individuo y la frase objetivo
    ncoin = 0

    # Para cada caracter en el individuo comprobamos
    # si coincide con el caracter de la frase objetivo
    for caracter in range(len(individuo)):
        if individuo[caracter] == frase_objetivo[caracter]:
            ncoin += 1

    # Número de caracteres de la frase objetivo
    longitud_tar = len(frase_objetivo)

    # Calculamos el fitnes (parecido) entre el individuo y la frase objetivo
    fitness = math.exp(ncoin - longitud_tar) - math.exp(-longitud_tar)

    return fitness, ncoin


def mutar_gen(gen: str, Pm: float) -> str:
    """
    Decide si un gen debe mutar en función de la probabilidad Pm.

    Si el gen muta, elegimos un nuevo al azar de la lista de genes.
    Si no muta, devolvemos el gen original.

    Args:
        gen (str): Carácter del individuo (Mayúscula o espacio).
        Pm (float): Probabilidad de que el gen mute (0 < Pm < 1).

    Returns:
        str: Nuevo gen (puede estar mutado o no).
    """

    # Elegimos una probabilidad al azar entre 0 y 1.
    # Si la probabilidad es menor que Pm, el gen muta.
    # Si no, devolvemos el gen original.
    probabilidad = random.random()

    if (probabilidad < Pm):
        # Elegimos un nuevo gen al azar de la lista de genes
        # y lo devolvemos.
        return random.choice(GENES)
    else:
        # Devolvemos el gen original.
        return gen


def mutar_individuo(padre, Pm):
    """
    Aplica una mutuación a cda gen de un individuo.

    Recorre todos los genes (caracteres) del individuo padre y, para cada uno,
    decide si debe mutar utilizando la función `mutar_gen`.

    Args:
        padre (str): String original
        Pm (float): Probabilidad de mutación (0 < Pm < 1).

    Returns:
        str: Nuevo individuo tras aplicar las mutaciones.
    """

    # Creamos un string vacío para el individuo mutado
    individuo_mutado = ""

    # Recorremos cada gen del individuo padre y aplicamos la mutación
    for gen in padre:
        individuo_mutado += mutar_gen(gen, Pm)

    return individuo_mutado


def obtener_consenso(poblacion):
    """
    Calcula el individuo consenso de una población.

    El inidividuo consenso es un string que contiene el
    caracter más frecuente en cada posición de la población.

    Args:
        poblacion (list): Lista de individuos (strings).

    Returns:
        str: Cadena de consenso construida con los caracteres más
        frecuentes en cada posición.
    """

    # Inicializamos el string para el individuo consenso
    # y recorremos cada posición de la población
    consenso = ""

    # Como todos los individuos tienen la misma longitud, podemos
    # usar la longitud del primer individuo como referencia
    # para recorrer la población.
    for posicion in range(len(poblacion[0])):

        # Cogemos la letra de cada inviduo en la posición actual
        letras = [individuo[posicion] for individuo in poblacion]

        # Contamos cuantas veces aparace cada letra y nos quedamos
        # con la letra que más veces aparece y la añadimos al consenso.
        # Counter(letras) -> Contamos las letras
        # (1) -> Seleccionamos la letra más común -> [tupla(letra, frecuencia)]
        # [0] -> Seleccionamos la tupla -> tupla(letra, frecuencia)
        # [0] -> Seleccionamos la letra de la tupla -> letra
        consenso += Counter(letras).most_common(1)[0][0]

    return consenso


# Algoritmo genético principal
def algoritmo_genetico(target, NPOB, NGEN, Pm, NRES, NSAMPLE):
    """
    Algoritmo genético para encontrar una frase objetivo

    Args:
        target (str): Frase objetivo que queremos alcanzar mediante evolución.
        NPOB (int): Número de individuos en la población.
        NGEN (int): Número total de generaciones a ejecutar.
        Pm (float): Probabilidad de mutación aplicada a cada gen de un individuo.
        NRES (int): Frecuencia (en generaciones) con la que se imprime un resumen estadístico.
        NSAMPLE (int): Frecuencia (en generaciones) con la que se muestra una muestra aleatoria de la población.

    Returns:
        list: Lista de individuos (strings) que representan la población final.
    """


    ltar = len(target)
    poblacion = [crear_individuo(ltar) for _ in range(NPOB)]
    fitnesses = []
    for ind in poblacion:
        fit, _ = calcular_fitness(ind, target)
        fitnesses.append(fit)

    print(f"TARGET: {target}, ltar: {ltar}, NPOB: {NPOB}, NGEN: {NGEN}, Pm: {Pm}")

    for gen in range(NGEN):
        ncoin_max = max(sum(1 for a, b in zip(ind, target) if a == b) for ind in poblacion)
        fit_max = math.exp(ncoin_max - ltar) - math.exp(-ltar)

        for _ in range(NPOB):
            i_padre = random.randint(0, NPOB - 1)
            fit_i, _ = calcular_fitness(poblacion[i_padre], target)
            Ps = fit_i / fit_max if fit_max != 0 else 0
            if random.random() < Ps:
                hijo = mutar_individuo(poblacion[i_padre], Pm)
                fit_hijo, _ = calcular_fitness(hijo, target)
                i_borrar = random.randint(0, NPOB - 1)
                poblacion[i_borrar] = hijo

        if gen % NRES == 0 or gen % NSAMPLE == 0:
            fitnesses = [calcular_fitness(ind, target)[0] for ind in poblacion]
            ncoin_list = [sum(1 for a, b in zip(ind, target) if a == b) for ind in poblacion]
            ncoin_max = max(ncoin_list)
            NTar = ncoin_list.count(ltar)
            pNTar = 100 * NTar / NPOB
            consenso = obtener_consenso(poblacion)
            ncoin_media = np.mean(ncoin_list)
            print(f"\nGen {gen}: Mejor individuo con {ncoin_max} coincidencias, NTar = {NTar} ({pNTar:.2f}%)")
            print(f"Consenso: {consenso}, Coincidencias medias: {ncoin_media:.2f}")

        if gen % NSAMPLE == 0:
            sample = random.sample(poblacion, NPOB // 5)
            print("\nSample poblacional:")
            for s in sample:
                print(s)

    return poblacion

# Parámetros de entrada
target = "DARTH VADER"       # Frase objetivo (puede tener cualquier longitud)
NPOB = 100                  # Número de individuos en la población
NGEN = 300                  # Número de generaciones a ejecutar
Pm = 0.1                    # Probabilidad de mutación
NRES = 20                   # Frecuencia de resumen (cada NRES generaciones)
NSAMPLE = 50               # Frecuencia de muestreo (cada NSAMPLE generaciones)

# Ejecutar el algoritmo genético
resultado = algoritmo_genetico(target, NPOB, NGEN, Pm, NRES, NSAMPLE)

TARGET: DARTH VADER, ltar: 11, NPOB: 100, NGEN: 300, Pm: 0.1

Gen 0: Mejor individuo con 2 coincidencias, NTar = 0 (0.00%)
Consenso: EUJMFICPSIO, Coincidencias medias: 0.43

Sample poblacional:
GTHIGAECLPY
UXOZMVOPCWO
FUNHKIJQOWZ
TTGXSIDGB S
NNKVUOEGBZZ
GUJEH  WTIO
Q KSSVUQZLV
DXOEVGIQVGV
WT QEYGXMTO
HAW OBTKLOE
JLZWWSNPPAY
BBZSDCYNPXN
OEHAHQUIFNW
KN UZIHGSAS
YSELHGCOGHV
RTHFAKRBFKM
MNHCKFJEPPP
MKSNRJAPONH
XDOC XGZOPO
TKCUBBIYVFQ

Gen 20: Mejor individuo con 4 coincidencias, NTar = 0 (0.00%)
Consenso: DHDOFAVAD O, Coincidencias medias: 3.16

Gen 40: Mejor individuo con 7 coincidencias, NTar = 0 (0.00%)
Consenso: DHROF VADED, Coincidencias medias: 4.58

Gen 50: Mejor individuo con 7 coincidencias, NTar = 0 (0.00%)
Consenso: DMRTZ VADEX, Coincidencias medias: 6.00

Sample poblacional:
DMRQZIVWDEP
DURHU VADEZ
DKRQZ AADE 
DTRTN VXDJK
DWRTN VYDEI
DGRQOYVSDSX
DRROZ VADEX
DTRTNEVGDED
DMJTG  ADEL
DRROZ VDDEX
DMJTZ VADEQ
DRROZ VADEX
DUOQU DADEX
DTRTN VYDED
TMJTZ VADEL
DZZMU DADEJ
DTRZN VYDED
D 